In [125]:
from twlogin import login
from tools import blockclock, precio , btc_supply, hash_rate, fees
import numpy as np

p1 = ''
api = login(path=p1)	


def execute(order):
   txt = order._json['full_text'].lower()
   txt_split= txt.split(" ")    #cambia este valor segun el nombre @user
   for word in txt_split:
      if word in ['blockclock','btcclock','btc_clock','clock','tic']:
         api.update_status(status='@'+order._json['user']['screen_name']+' '+blockclock(),in_reply_to_status_id=order._json['id_str'],auto_populate_reply_metadata=True)
         return order._json['id_str']
      elif word in ['btc_supply','supply','emitido','btc_emitido','btc_emision','emision','emision_btc']:
         api.update_status(status='@'+order._json['user']['screen_name']+' '+btc_supply(),in_reply_to_status_id=order._json['id_str'],auto_populate_reply_metadata=True)
         return order._json['id_str']
      elif word in ['hash_rate','hash-rate']:
         api.update_status(status='@'+order._json['user']['screen_name']+' '+hash_rate(),in_reply_to_status_id=order._json['id_str'],auto_populate_reply_metadata=True)
         return order._json['id_str']
      elif word == 'fees':
         api.update_status(status='@'+order._json['user']['screen_name']+' '+fees(),in_reply_to_status_id=order._json['id_str'],auto_populate_reply_metadata=True)
         return order._json['id_str']
      elif word in ['precio','btc_price','bitcoin_precio']:
         api.update_status(status='@'+order._json['user']['screen_name']+' '+precio(),in_reply_to_status_id=order._json['id_str'],auto_populate_reply_metadata=True)
         return order._json['id_str']
         #retorna el id donde responde, o False en caso de que no haya un comando.
   return False



# A diferencia de otros métodos, este obtiene las ultimas 20 menciones
# incluidos rt sin Cursor. Se puede modificar la recuperación de 20 dependiento de 
# la fecha de inicio y fin de la búsqueda. 

# Para controlar un flujo y evitar responder menciones repetidas, usamos un archivo
# externo para almacenar el id de las menciones anteriores. Por facilidad en recuperar
# y escribir un log usamos numpy y .npy 

def first_up(path=''):
   #la primera vez que se ejecuta el bot, verifica si existe un log.
   #En caso de que no, realiza el analisis:
   ids = [] #en esta variable guardamos id de respuestas para el log
   #recopila las últimas 20 menciones
   historial = api.mentions_timeline(tweet_mode='extended')
   #verifica que historial no sea una lista vacia
   if len(historial) == 0:
      #en caso de que sí, crea el log vacío.
      np.save(path+'respuestas.npy', ids)
   else:
      #en caso de que hayan meciones 
      for order in historial[::-1]:
         aux = execute(order)
         if not aux:
            pass
         else: 
            ids.append(aux)
   #finalmente guardamos el log 
      np.save(path+'respuestas.npy', ids)
   



#verificamos si existe un log
if not os.path.exists(p1+'respuestas.npy'):
   # primer analisis si es que no
   first_up(path=p1) 
else:
   # resp cuando ya existe puede ser estar vacio eventualmente
   # para nuevas menciones solo se necesita rescatar
   # el valor mas reciente de la lista y solo procesará
   # nuevas menciones. 
   resp = np.load(p1+'respuestas.npy',allow_pickle='TRUE').tolist()
   if not resp: #podria pasar que resp sea una lista vacia
      #en este caso llamamos a la api igual que en first_up
      historial = api.mentions_timeline(tweet_mode='extended')
      if len(historial)==0:  #si no hay nada, no pasa nada
         pass
      else:
         for order in historial[::-1]:
            aux = execute(order)
            if aux:
               resp.append(aux)
            else:
               pass
   else: 
      #recuperamos la ultima mención del log para iniciar desde ese id para adelante
      #puede darse el caso de que las menciones sean >20 por lo que se deja al bot en bucle
      #para q vaya procesando por bloques de 20.
      historial = api.mentions_timeline(tweet_mode='extended',since_id=resp[-1])
      for order in historial[::-1]:
         resp.append(execute(order))   
   np.save(p1+'respuestas.npy', resp)

autentificación - Twitter API
sesión iniciada


In [127]:
import numpy as np

test = np.load('respuestas.npy',allow_pickle='TRUE').tolist()
print(test)

['1547779084070952964', '1547780686429315074']


In [106]:
from twlogin import login
from tools import blockclock, precio , btc_supply, hash_rate, fees
import numpy as np

p1 = ''
api = login(path=p1)	
historial = api.mentions_timeline(tweet_mode='extended')

autentificación - Twitter API
sesión iniciada


In [107]:
historial[0]

Status(_api=<tweepy.api.API object at 0x0000019555D25700>, _json={'created_at': 'Fri Jul 15 02:56:49 +0000 2022', 'id': 1547777173573947392, 'id_str': '1547777173573947392', 'full_text': '@elnodobtc @marcel039 @elnodobtc blockclock', 'truncated': False, 'display_text_range': [22, 43], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'elnodobtc', 'name': '🇧🇴⚡₿it₿ol-node | in ⌚⛓️7️⃣4️⃣4️⃣7️⃣9️⃣2️⃣', 'id': 1543984303390048256, 'id_str': '1543984303390048256', 'indices': [0, 10]}, {'screen_name': 'marcel039', 'name': '🇦🇷⚡️Marcel 侘寂 Free', 'id': 31786786, 'id_str': '31786786', 'indices': [11, 21]}, {'screen_name': 'elnodobtc', 'name': '🇧🇴⚡₿it₿ol-node | in ⌚⛓️7️⃣4️⃣4️⃣7️⃣9️⃣2️⃣', 'id': 1543984303390048256, 'id_str': '1543984303390048256', 'indices': [22, 32]}], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1547776594067935232, 'in_reply_to_status_id_str': '1547776594067935232', 'in_reply

In [108]:
kk =historial[0]._json['user']['screen_name']

In [109]:
api.update_status(status='@'+kk+' jeje',in_reply_to_status_id=historial[0]._json['id_str'],auto_populate_reply_metadata=True)


Status(_api=<tweepy.api.API object at 0x0000019555D25700>, _json={'created_at': 'Fri Jul 15 02:57:47 +0000 2022', 'id': 1547777417019764736, 'id_str': '1547777417019764736', 'text': '@jpcr3spo @marcel039 @jpcr3spo jeje', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'jpcr3spo', 'name': 'juan.py ⚡', 'id': 1416223413052530696, 'id_str': '1416223413052530696', 'indices': [0, 9]}, {'screen_name': 'marcel039', 'name': '🇦🇷⚡️Marcel 侘寂 Free', 'id': 31786786, 'id_str': '31786786', 'indices': [10, 20]}, {'screen_name': 'jpcr3spo', 'name': 'juan.py ⚡', 'id': 1416223413052530696, 'id_str': '1416223413052530696', 'indices': [21, 30]}], 'urls': []}, 'source': '<a href="https://www.youtube.com/" rel="nofollow">bolivian_node</a>', 'in_reply_to_status_id': 1547777173573947392, 'in_reply_to_status_id_str': '1547777173573947392', 'in_reply_to_user_id': 1416223413052530696, 'in_reply_to_user_id_str': '1416223413052530696', 'in_reply_to_screen_name': 'jpc